In [2]:
import ast
import json
import os

import pandas as pd
from dotenv import load_dotenv

load_dotenv()  # Take environment variables from .env
api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI

client = OpenAI(
    base_url="http://localhost:1234/v1",  # local model
    api_key=api_key
)

In [39]:
# Read the CSV
df = pd.read_csv("data/output/processed_ingredients.csv")
# Parse the JSON-style ingredient lists
df["ingredients_normalized"] = df["ingredients_normalized"].apply(json.loads)
# Ensure they're lists
df["ingredients_list"] = df["ingredients_normalized"].apply(
    lambda x: ast.literal_eval(str(x)) if isinstance(x, str) else x
)
# Flatten all lists and deduplicate
unique_ingredients = sorted({item for sublist in df["ingredients_list"] for item in sublist})

Saved 2019 unique ingredients to unique_ingredients.txt


In [ ]:
# Save to a text file, one ingredient per line
with open("/checkpoints/unique_ingredients.txt", "w", encoding="utf-8") as f:
    f.write("\n".join(unique_ingredients))

print(f"Saved {len(unique_ingredients)} unique ingredients to unique_ingredients.txt")

In [ ]:
gpt_prompt = """
Remove duplicates and anything that isn't an ingredient.
"""
model = "qwen/qwen3-4b-2507"

In [ ]:
# reinforce JSON-only on the user message
response = client.chat.completions.create(
    model=model,
    messages=[
        {"role": "system", "content": gpt_prompt},
    ],
)

In [ ]:
response